# Examen NLP avec python 2023

### Sujet

L'objectif de cet examen est d'analyser un corpus de textes et de le catégoriser par thématique. 

Le corpus est 20 newsgroup. Il est composé de posts de média répartis en 20 catégories différentes. 
Les données sont labellisées. 

Vous allez utiliser une approche supervisée et une approche non supervisée pour cette tâche. 

Pour la partie supervisée, vous allez utiliser un algorithme de classification de texte. Les labels fournis permettront d'apprendre à catégoriser les posts en fonction de leur thématique.

Pour la partie non-supervisée, vous allez utiliser une technique de topic modeling pour découvrir des catégories (clusters) parmi les données sans utiliser les labels fournis. Vous utiliserez uniquement le nombre de catégories existantes, qui sera le nombre de clusters que vous cherchez. 

Chacune des techniques nécessitera une étape de vectorisation et sera évaluée avec une métrique adaptée. 



---



### Consignes

Le fichier peut être exécuté au choix sur Colab, Kaggle ou sur votre envrionnement local. Colab et Kaggle sont fortement conseillés pour limiter les problèmes liés à l'installation de librairies. De plus, Le GPU peut être utilisé sur Kaggle pour limiter les temps de calcul (surtout pour l'utilisation de grands modèles).

Toutes les cellules comprenant l'emoji ➡️ sont des consignes avec différentes tâches à réaliser. 

Les cellules vides doivent être complétées. Vous pouvez ajouter autant de cellules que vous le souhaitez.

Vous n'êtes pas obligé·e·s de faire l'examen dans l'ordre. Cependant il doit être rendu dans l'ordre fourni au début et il sera exécuté dans cet ordre pour la correction. 

Attention à la mémoire tampon des notebooks. Si vous rencontrer des problèmes pendant l'examen, pensez à redémarrer l'environnement d'exécution. De même, il est conseillé d'essayer de redémarrer l'environnement d'exécution et de relancer le code avant de rendre le devoir pour vérifier que tout fonctionne.

Il est vivement recommandé d'utiliser les librairies existantes pour réaliser les différentes tâches demandées. Cependant, tous les choix de paramètres doivent être motivés. Si des paramètres inutiles ou non expliqués sont présents dans le code, cela sera pris en compte dans la notation.




---



⭐ Barème pour les intervenant·e·s : 

Le barème est proposé pour une note sur 20. 

Il est pûrement indicatif afin d'harmoniser le traitement des travaux entre les campus. 

Une partie des points peut être donnée si les choix d'implémentation sont bons mais que le code plante. 
De même, une partie des points peut être retirée si le code tourne mais qu'il n'est pas propre, que les noms de variables ne sont pas explicites, que des lignes sont dupliquées ou inutiles, etc...



## Import des packages

➡️ Ajouter les packages nécessaires au fur et à mesure de l'examen.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

## Etude des données

### Chargement

➡️ Exécuter le code suivant pour charger les données.

In [ ]:
news_dataset = fetch_20newsgroups(subset='all')

### Caractéristiques

➡️ Afficher la taille du dataset

In [ ]:
# correction
print(f"Taille du dataset : {len(news_dataset.data)}")

⭐ Barème : 1 point

➡️ Afficher le nom des catégories du dataset.

In [ ]:
# correction
news_dataset.target_names

⭐ Barème : 1 point

## Vectorisation

➡️ Séparer les données en un jeu d'entraînement et un jeu de test en réduisant le jeu d'entraînement à 1000 exemples et le jeu de test à 250 exemples pour limiter les temps de calcul.

Attentions à répartir équitablement les classes.

In [ ]:
# correction sklearn train_test_split
from sklearn.model_selection import train_test_split

X_train_news, x_test_news, y_train_news, y_test_news = train_test_split(news_dataset.data, news_dataset.target, train_size=1000, test_size=250, shuffle=True, stratify=news_dataset.target)

⭐ Barème : 3 points

- découpage et nommage des variables (1 point)
- respect des proportions (1 point)
- utilisation de `stratify` (1 point)

➡️ Choisir une méthode de vectorisation adaptée à la tâche à réaliser. 

➡️ Implémenter cette méthode et vectoriser l'ensemble des données.

➡️ Justifier le choix de la méthode.

In [ ]:
# correction BOW
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train_news)
x_test_bow = bow_vectorizer.transform(x_test_news)

In [ ]:
# correction TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_news)
x_test_tfidf = tfidf_vectorizer.transform(x_test_news)

In [ ]:
# correction Bert
! pip install transformers
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

last_hidden_state_index = 0
batch_index = 0
cls_token_index = 0

def encode_text(text):
  encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
  output = model(**encoded_input)
  cls_token = output[last_hidden_state_index][batch_index][cls_token_index].tolist()
  return cls_token

X_train_bert = []
for utterance in X_train_news:
  X_train_bert.append(encode_text(utterance))

x_test_bert = []
for utterance in x_test_news:
  x_test_bert.append(encode_text(utterance))

⭐ Barème : 5 points 

- choix de la méthode adaptée (1 point)
- justification pertinente (1 point)
- pour BOW ou TF-IDF :
  - instanciation du modèle sans paramètre inutile (1 point)
  - utilisation de la méthode `fit_transform` sur les données d'entraînement (1 point)
  - utilisation de la méthode `transform` sur les données de test (1 point)
- pour les méthodes de type embeddings contextualisés, Bert et autres :
  - choix du modèle pertinent (1 point)
  - technique de vectorisation (1 point)
  - utilisation du token cls ou autre à justifier (1 point)
- à adapter pour toute autre méthode de vectorisation

## Classification supervisée

➡️ Choisir un algorithme vous paraissant pertinent pour la tâche de catégorisation de texte.

➡️ Entraîner le modèle grâce au jeu d'entraînement.

In [ ]:
# correction
from sklearn import svm

classif_model_tfidf = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',probability=True)    
classif_model_tfidf.fit(X_train_tfidf,y_train_news)

⭐ Barème 3 points: 

- choix du modèle de classification pertinent (1 point)
- utilisation de la méthode `fit` sur les données d'entraînement (2 points)

➡️ Utiliser le modèle entraîné pour faire les prédictions correspondantes au jeu de test. 

➡️ Afficher les performances du modèle. 

In [ ]:
# correction
y_pred_tfidf = classif_model_tfidf.predict(x_test_tfidf)

In [ ]:
# correction
from sklearn.metrics import classification_report

print(classification_report(y_test_news, y_pred_tfidf))

⭐ Barème : 2 points

- utilisation de la méthode `predict` sur les données de test (1 point)
- affichaque d'une métrique pertinente telle que l'accuracy ou f1-score (1 point)

➡️ Recommencer la partie classification avec SOIT :
- une autre technique de vectorisation de votre choix
- une étape de pre-processing à appliquer avant de reproduire la même vectorisation. 

Les jeux d'entraînement et de test doivent rester les mêmes pour que la comparaison soit la plus pertinente possible. Le modèle de classification choisi ne doit pas changer non plus, seule la technique de vectorisation ou l'ajout de l'étape de pre-processing change. 

➡️ Commenter les résultats obtenus. Tenter d'expliquer la différence de performance s'il y en a une.

In [ ]:
# correction autre modèle
classif_model_bert = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',probability=True)    

classif_model_bert.fit(X_train_bert,y_train_news)
y_pred_bert = classif_model_bert.predict(x_test_bert)
print(classification_report(y_test_news, y_pred_bert))

In [ ]:
# correction pre-processing

def clean_text(text):
  clean_text = text.replace('\n','')
  clean_text = clean_text.replace('\t','')
  # tri des emails, des stop words, de la ponctuation, lemmatisation, etc... 
  return clean_text

X_train_pre = [clean_text(news) for news in X_train_news]
x_test_pre = [clean_text(news) for news in x_test_news]

tfidf_vectorizer = TfidfVectorizer()
X_train_pre_tfidf = tfidf_vectorizer.fit_transform(X_train_pre)
x_test_pre_tfidf = tfidf_vectorizer.transform(x_test_pre)

classif_model_pre_tfidf = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',probability=True)    
classif_model_pre_tfidf.fit(X_train_pre_tfidf,y_train_news)
y_pred_pre_tfidf = classif_model_pre_tfidf.predict(x_test_pre_tfidf)
print(classification_report(y_test_news, y_pred_pre_tfidf))

⭐ Barème : 5 points

- changement de technique de vectorisation : 2 points
  - choix d'une technique pertinente (1 point)
  - bonne implémentation sur les mêmes données (1 point)
- pre-processing sur la même technique de vectorisation : 2 points
  - les techniques de pre-processing sont choisies de façon pertinente par rapport aux données (1 point)
  - le pre-processing est implémenté proprement avec les librairies adaptées comme nltk, spacy, en utilisant des regex, la fonction replace, etc... (1 point)
- nouvelle classification correctement exécutée (1 point)
- explication sur la différence de performance (2 points)

## Topic Modeling

➡️ Choisir une méthode **non supervisée** permettant de faire du topic modeling. Commenter votre choix.

➡️ Implémenter cette méthode et l'exécuter sur les données vectorisées avec une des méthodes précédemment utilisée de votre choix.

Le nombre de clusters est fixé à **20**.

In [ ]:
# correction
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=20)
lda.fit(X_train_tfidf)
y_pred_lda = lda.transform(x_test_tfidf)

⭐ Barème : 3 points

- choix de la méthode pertinente comme LDA, LSA, K-means,... (1 point)
- bonne implémentation de la méthode (2 points)

➡️ Afficher les mots représentatifs de chacun des clusters.

In [ ]:
# correction
for index,topic in enumerate(lda.components_):
    print(f'10 mots majoritairement représentant le cluster {index} : ')
    print([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

⭐ Barème : 1 point

➡️ Utiliser une métrique vous paraissant pertinente pour évaluer les clusters identifiés par rapport aux labels existants des données. 

➡️ Commenter le score obtenu.

In [ ]:
# correction
import numpy as np
from sklearn.metrics.cluster import adjusted_rand_score

y_pred_clusters = []
for pred in y_pred_lda:
  y_pred_clusters.append(np.argmax(pred))

adjusted_rand_score(y_test_news, y_pred_clusters)

⭐ Barème : 1 point